## Config

In [1]:
!pip install tensorflow

In [2]:
%tensorflow_version 2.x

In [3]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

## Dataset

In [5]:
CSV_COLUMN_NAMES = ['SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species'] # no entiendo por qué entiende que hay 4 campos
SPECIES = ['Setosa', 'Versicolor', 'Virginica'] # 3 especies distintas, que serán las clases de nuestro modelo (ver más adelante)

In [7]:
# Descarga de los ficheros CSV
trainPath = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv" )
testPath = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# uso del módulo keras para leer un csv y transformarlo en un 'pandas dataset' (train & test variables)
train = pd.read_csv(trainPath, names=CSV_COLUMN_NAMES, header=0) # header = 0 porque la fila 0 es la cabecera
test = pd.read_csv(testPath, names=CSV_COLUMN_NAMES, header=0)

In [ ]:
train.head() # comprobación

,SepalLenght,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


Separamos el campo 'Species' para usarlo como Label

In [8]:
trainY = train.pop('Species') # label
testY = test.pop('Species')

train.head() # vemos cómo queda el dataset

,SepalLenght,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [9]:
trainY.head() # especies del 0 al 2

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [10]:
train.shape # shape es una propiedad del dataset no un método
# 120,4 significa-> 120 datos, 4 elementos por cada dato

(120, 4)

Definición de la **Input Function**
- Aclaración: en esta ocasión, no hemos hecho una función que devuelve otra función. 

In [11]:
def inputFunction(features, labels, training=True, batchSize=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels)) # transforma los inputs de pandas dataset a tensorflow.data.DataSet object
  if training: # si estamos en modo entrenamiento, randomizar y repetir (una vez?)
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batchSize)

Feature Columns

In [12]:
# Feature columns describe how to use the input.
# las feature columns de TensorFlow contienen la información necesaria para que la input function sepa cómo procesar la información 
featureColumns = []
for key in train.keys(): # keys devuelve el "info axis" del dataset, es decir los nombres de las columnas
 featureColumns.append(tf.feature_column.numeric_column(key=key))

print(featureColumns)

[NumericColumn(key='SepalLenght', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


Creación del modelo
- Elegimos DNN (Deep Neural Network) (lo estudiaremos más adelante)
  - Éste tendrá 2 'hidden layers'
    - Una con 30 'hidden nodes' (los nodos también se llaman neuronas)
    - Otra con 10 'hidden nodes'
  - 3 clases (una para cada especie)

In [13]:
classifier = tf.estimator.DNNClassifier( # creación del modelo DNN
    feature_columns=featureColumns, # inputs
    hidden_units=[30,10], # 2 capas, cada una con 30 y 10 nodos
    n_classes=3 # 3 clases una por cada especie
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppe7jg5hs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Entrenamiento del modelo
- steps: similares a los epochs: se pueden cambiar para obtener mayor o menor precisión.

In [14]:
classifier.train(
    input_fn=lambda: inputFunction(train, trainY, training=True), # sintaxis para pasar una función a otra función
    steps=8000
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmppe7jg5hs/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.9336848, step = 0
INFO:tensorflow:global_step/sec: 469.309
INFO:tensorflow:loss = 1.6667736, step = 100 (0.218 sec)
INFO:tensorflow:global_step/sec: 527.873
INFO:tensorflow:loss = 1.3248044, step = 200 (0.190 sec)
INFO:tensorflow:global_step/sec

Testeo/Evaluación del modelo
- Nota: al cambiar el nº de steps de 5000 a 8000, el modelo pasó de 53% de precisión a 93% de precisión!

In [15]:
evaluationResult = classifier.evaluate(
    input_fn=lambda: inputFunction(test, testY, training=False) # no ponemos nº de steps porque estamos evaluando el modelo con el dataset de test
)
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**evaluationResult))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-04T15:53:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppe7jg5hs/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22684s
INFO:tensorflow:Finished evaluation at 2021-05-04-15:53:06
INFO:tensorflow:Saving dict for global step 8000: accuracy = 0.8666667, average_loss = 0.55061936, global_step = 8000, loss = 0.55061936
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: /tmp/tmppe7jg5hs/model.ckpt-8000

Test set accuracy: 0.867



Predicciones
- Ahora usaremos el modelo entrenado para hacer predicciones. 
- Basado en la anchura y longitud de los pétalos el modelo intentará precedir cuál de las tres especies de flores tenemos entre manos
- El siguiente scripts utiliza el modelo que hemos creado y entrenado para, al darle una entrada (4 valores para una flor) nos diga qué especie es la más probable que sea.

In [16]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Aquí entendemos porqué se separan los lables (yesTrain/yesEval) que es la variable que queremos averiguar.
# En esta input function que usamos para testear el modelo sólo introduciremos, 1 registro (feature:value, feature2:value etc)
# Aunque indicamos que el batch_size es 256 en realidad sólo le vamos a dar un valor (la flor que mete el usuario)

features = ['SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth'] 
predict = {} # instanciación de un diccionario (clave:valor)

print("Please type numeric values as prompted.")
for feature in features: # itera sobre el features
  valid = True # válido por defecto
  while valid: # siempre que sea válido
    val = input(feature + ": ") # le pide al usuario con input
    if not val.isdigit(): valid = False # Si el valor introducido por el usuario no es un dígito-> valid pasa a valer false y se sale del bucle

  predict[feature] = [float(val)] 
  # almacena en el diccionario predict claves: 'SepalLenght', 'SepalWidth', 'PetalLength', 'PetalWidth' el valor para cada uno que ha introducido el suuario
  # vemos que el valor del usuario lo metemos en una lista -> []; 
  # esto es así porque el método predict() de TensorFlow, predice para múltiples valores, es decir, que para cada tamaño del pétalo o sépalo, en lugar de un solo valor
  # TensorFlow está esperando 1 o más valores de medición para luego hacer la predicción de qué especie es.

predictions = classifier.predict(input_fn=lambda: input_fn(predict)) # creamos la funcion introduciendo el diccionario que acabamos de crear con los valores del usuario
for pred_dict in predictions:
    print(pred_dict) # para que veamos el diccionario completo
    # Las siguientes 2 líneas están explicadas en el texto más abajo
    class_id = pred_dict['class_ids'][0] # En la key 'class_ids' del diccionario en la posicion 0 (la única que hay en realidad) se almacena cuál de las especies tiene más probabilidad de ser según el modelo de predicción
    probability = pred_dict['probabilities'][class_id] # la probabilidad de que sea la especie más probable (ojo trabalenguas)

    print('Prediction is "{}" ({:.1f}%)'.format( # 
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
SepalLenght: 0.2
SepalWidth: 0.4
PetalLength: 06.
PetalWidth: 0.42
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppe7jg5hs/model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.28884184,  1.4509197 ,  1.9730763 ], dtype=float32), 'probabilities': array([0.06135921, 0.3495011 , 0.5891397 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (58.9%)


Este es la pinta de uno de los diccionarios de prediccion que genera predict()  
{'logits': array([ 1.8688045, -0.6955673, -1.8335314], dtype=float32), 'probabilities': array([0.9077431 , 0.06986673, 0.02239025], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
- Vemos que en la key 'probabilities' tenemos 3 valores , uno para cada una de las diferentes clases. 
- En 'class_id': muestra cuál de las 3 especies es la que más probabilidades tiene de ser la especie.
- Si miramos cuando habíamos declarado las especies SPECIES = ['Setosa', 'Versicolor', 'Virginica']. 0, 1 y 2 serían Setosa, Versicolor y Virginica